<h1><center>HW2 Scrape Hotel Reviews</center></h1>

Choose one hotel at tripadvisor.com (e.g.https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html)

- Q1. Write a function to scrape all **reviews on the first page**, including, 
    - **username** (see (1) in Figure)
    - **review date** (see (2) in Figure)
    - **rating** (see (3) in Figure)
    - **title** (see (4) in Figure)
    - **review text** (see (5) in Figure. For each review text, need to get the **complete text**.)
    - **date of stay**(see (6) in Figure)
    - If a field, e.g. rating, is missing, use `None` to indicate it. 
- `Function Input`: hotel page URL
- `Function Output`: save all reviews as a DataFrame of columns. E.g., for the given URL, you can get 10 reviews.

    
![alt text](tripadvisor.png "TripAdvisor")

In [157]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # deal with deprecation warnings

import requests
from bs4 import BeautifulSoup  
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re


In [153]:
def getReviews(page_url):

    driver = webdriver.Safari()
    page_url = page_url + '#REVIEWS' # get the reviews tab
    driver.get(page_url)
    time.sleep(3)
    reviews_tb = pd.DataFrame(columns=['username', 'review_date', 'rating', 'title', 'review_text', 'date_of_stay'])

    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.YibKl.MC.R2.Gi.z.Z.BB.pBbQr')

    for i, review in enumerate(reviews):

        try:
            if i == 0: # clicks on the first review's "Read more" button
                       # during debugging found out that this seems to expand all the reviews on the page
                button = review.find_element(By.CSS_SELECTOR, 'div.TnInx') # read more button
                button.click()
                time.sleep(5)

            # username and review_date
            head_data = review.find_element(By.CSS_SELECTOR, 'div.cRVSd').text
            head_data = head_data.split('wrote a review')
        
            username = head_data[0].strip()
            review_date = head_data[-1].strip()


            # rating
            rating = review.find_element(By.CSS_SELECTOR, 'span.ui_bubble_rating').get_attribute('class')
            rating = rating.replace('ui_bubble_rating bubble_', '')
            rating = rating.replace('0', '').strip()
            rating = float(rating)

            # title
            title = review.find_element(By.CSS_SELECTOR, 'a.Qwuub').text

            

            # review_text
            review_text = review.find_element(By.CSS_SELECTOR, 'div.fIrGe._T').text

            # date_of_stay
            date_of_stay = review.find_element(By.CSS_SELECTOR, 'span.teHYY._R.Me.S4.H3').text
            date_of_stay = date_of_stay.replace('Date of stay: ', '').strip()

            # Add to the dataframe
            reviews_tb = reviews_tb.append({'username': username, 'review_date': review_date, \
                                'rating': rating, 'title': title, 'review_text': review_text, \
                                                'date_of_stay': date_of_stay}, ignore_index=True)

            time.sleep(1)

        except Exception as e:
            print(e)
            continue

    driver.quit()



    

    return reviews_tb


In [154]:
page_url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html'
getReviews(page_url)

,username,review_date,rating,title,review_text,date_of_stay
0,uk,Yesterday,4.0,The saviour of our NYC trip,We originally had booked to stay at a differen...,February 2023
1,Alice L,Feb 2022,5.0,Perfect for Us,We recently chose Motto for an overnight in NY...,February 2022
2,Jonathan F,Yesterday,5.0,"Pleasant, value hotel smack in the middle of N...",Just a really great hotel. Achieved a lot with...,February 2023
3,Kristina D,Yesterday,5.0,Very friendly staff,I stayed at motto for 6 nights!The rooms were ...,February 2023
4,scott w,Feb 19,5.0,"Great room, great service, great location",Room was fantastic and the service was outstan...,February 2023
5,Louis M,Feb 19,5.0,Motto NYC Chelsea,We had an excellent one-night stay at the Mott...,February 2023
6,Brady,Feb 19,3.0,Whats in a name?,Its the Hilton name so booked on that assumpti...,February 2023
7,Robsen,Feb 17,5.0,Just perfect!,At first the arrival there was amazing. Ina fr...,February 2023
8,Kristin N,Feb 16,5.0,"Great Hotel, Wonderful Customer Service!",I've stayed at the Motto hotel several times o...,February 2023
9,koldoVergara_77,Feb 16,5.0,amazing!,happy to be back to NY after 16 years!Great We...,February 2023


In [217]:
page_url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html'
getReviews(page_url)


,username,review_date,rating,title,review_text,date_of_stay
0,Vic S,Today,5.0,First Choice Every Time,We have stayed at this hotel multiple times in...,February 2023
1,Alice L,Feb 2022,5.0,Perfect for Us,We recently chose Motto for an overnight in NY...,February 2022
2,Kari C,Today,5.0,Fabulous Hotel,We recently stayed at the Motto by Hilton in C...,January 2023
3,SamBBer,Today,5.0,Beautiful place & perfect location,We like to enjoy a weekend in NYC from time to...,December 2022
4,joel l,Yesterday,3.0,"No closet, no desk, no dresser",I understand that hotel rooms are often on the...,February 2023
5,lauryn r,Yesterday,5.0,Yasmin is amazing!,Yasmin was absolutely amazing!!! She really he...,January 2023
6,Tom I,Feb 2,5.0,Amazing location and view.,"Motto Chelsea was just amazing, the help of Ze...",January 2023
7,Passport58331943467,Feb 2,5.0,Amazing property and staff!,I booked a stay to celebrate my partner’s birt...,January 2023
8,Wade4020,Feb 2,5.0,Just a great comfortable experience.,The last few times I have traveled to NYC I ha...,February 2023
9,Paradise99206,Feb 1,4.0,Awesome location,A great location. Rooms were clean and functio...,January 2023


In [212]:
def getReviews2(page_url, num_pages=5):
    
    driver = webdriver.Safari()
    page_url = page_url + '#REVIEWS' # get the reviews tab
    driver.get(page_url)
    time.sleep(3)
    reviews_tb = pd.DataFrame(columns=['username', 'review_date', 'rating', 'title', 'review_text', 'date_of_stay'])

    for i in range(num_pages):

        # getReviews()
        reviews = driver.find_elements(By.CSS_SELECTOR, 'div.YibKl.MC.R2.Gi.z.Z.BB.pBbQr')

        for j, review in enumerate(reviews):

            try:
                if j == 0: # clicks on the first review's "Read more" button
                        # during debugging found out that this seems to expand all the reviews on the page
                    button = review.find_element(By.CSS_SELECTOR, 'div.TnInx') # read more button
                    button.click()
                    time.sleep(5)

                # username and review_date
                head_data = review.find_element(By.CSS_SELECTOR, 'div.cRVSd').text
                head_data = head_data.split('wrote a review')
            
                username = head_data[0].strip()
                review_date = head_data[-1].strip()


                # rating
                rating = review.find_element(By.CSS_SELECTOR, 'span.ui_bubble_rating').get_attribute('class')
                rating = rating.replace('ui_bubble_rating bubble_', '')
                rating = rating.replace('0', '').strip()
                rating = float(rating)

                # title
                title = review.find_element(By.CSS_SELECTOR, 'a.Qwuub').text

                

                # review_text
                review_text = review.find_element(By.CSS_SELECTOR, 'div.fIrGe._T').text

                # date_of_stay
                date_of_stay = review.find_element(By.CSS_SELECTOR, 'span.teHYY._R.Me.S4.H3').text
                date_of_stay = date_of_stay.replace('Date of stay: ', '').strip()

                # Add to the dataframe
                print(f"Adding page {i+1}, review {j+1} to dataframe")
                reviews_tb = reviews_tb.append({'username': username, 'review_date': review_date, \
                                    'rating': rating, 'title': title, 'review_text': review_text, \
                                                    'date_of_stay': date_of_stay}, ignore_index=True)

                time.sleep(1)

            except Exception as e:
                print(e)
                continue
            
        # end getReviews()

        page_url = f'https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-or{i+1}0-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html#REVIEWS'
        driver.get(page_url)
        time.sleep(3)
        
        
    # end for loop

    driver.quit()
    
    
    return reviews_tb

    

In [213]:
page_url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html'
df = getReviews2(page_url)

df.to_csv('reviews.csv', index=False)


Adding page 1, review 1 to dataframe
Adding page 1, review 2 to dataframe
Adding page 1, review 3 to dataframe
Adding page 1, review 4 to dataframe
Adding page 1, review 5 to dataframe
Adding page 1, review 6 to dataframe
Adding page 1, review 7 to dataframe
Adding page 1, review 8 to dataframe
Adding page 1, review 9 to dataframe
Adding page 1, review 10 to dataframe
Adding page 2, review 1 to dataframe
Adding page 2, review 2 to dataframe
Adding page 2, review 3 to dataframe
Adding page 2, review 4 to dataframe
Adding page 2, review 5 to dataframe
Adding page 2, review 6 to dataframe
Adding page 2, review 7 to dataframe
Adding page 2, review 8 to dataframe
Adding page 2, review 9 to dataframe
Adding page 2, review 10 to dataframe
Adding page 3, review 1 to dataframe
Adding page 3, review 2 to dataframe
Adding page 3, review 3 to dataframe
Adding page 3, review 4 to dataframe
Adding page 3, review 5 to dataframe
Adding page 3, review 6 to dataframe
Adding page 3, review 7 to dataframe

In [244]:
page_url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html'
getReviews2(page_url)

,username,review_date,rating,title,review_text,date_of_stay
0,Vic S,Today,5.0,First Choice Every Time,We have stayed at this hotel multiple times in...,February 2023
1,Alice L,Feb 2022,5.0,Perfect for Us,We recently chose Motto for an overnight in NY...,February 2022
2,Kari C,Today,5.0,Fabulous Hotel,We recently stayed at the Motto by Hilton in C...,January 2023
3,SamBBer,Today,5.0,Beautiful place & perfect location,We like to enjoy a weekend in NYC from time to...,December 2022
4,joel l,Yesterday,3.0,"No closet, no desk, no dresser",I understand that hotel rooms are often on the...,February 2023
5,lauryn r,Yesterday,5.0,Yasmin is amazing!,Yasmin was absolutely amazing!!! She really he...,January 2023
6,Tom I,Feb 2,5.0,Amazing location and view.,"Motto Chelsea was just amazing, the help of Ze...",January 2023
7,Passport58331943467,Feb 2,5.0,Amazing property and staff!,I booked a stay to celebrate my partner’s birt...,January 2023
8,Wade4020,Feb 2,5.0,Just a great comfortable experience.,The last few times I have traveled to NYC I ha...,February 2023
9,Paradise99206,Feb 1,4.0,Awesome location,A great location. Rooms were clean and functio...,January 2023
